# CNN

## Learning a Pattern
In this example, we will see how we can create a simple CNN that *learns to recognize an arbitrary pattern*.

Then we will see if such a network can learn also to repair a broken pattern.

Finally, we will see some applications of CNN to genomic medicine.